<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/HubSpot_Logo.svg/220px-HubSpot_Logo.svg.png" alt="drawing" width="200" align='left'/>

# Hubspot - Send sales brief
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Qonto/Qonto_Releve_de_compte_augmente.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

#hubspot #crm #sales #brief #email

## Input

### Import library

In [ ]:
from naas_drivers import emailbuilder
import naas
import pandas as pd
from datetime import datetime
from hubspot import Hubspot
hubspot = Hubspot()

### Enter your Hubspot api key

In [ ]:
auth_token = 'YOUR_HUBSPOT_API_KEY'

In [ ]:
auth_token = naas.secret.get("HUBSPOT_API_KEY")

### Connect to Hubspot

In [ ]:
hs = hubspot.connect(auth_token)

### Email parameters

In [ ]:
# Receivers
email_to = ["florent.ravenel@cashstory.com",
            "jeremy.ravenel@cashstory.com",
            "valentin.piquard@cashstory.com"]

# Email subject
email_subject = f"🚀 Hubspot - Sales Brief as of {datetime.now().strftime('%d/%m/%Y')} (Draft)"

### Sales target

In [ ]:
objective = 300000

### Schedule automation

In [ ]:
# naas.scheduler.add(cron="0 8 * * 1")

### Pick your pipeline

#### Get all pipelines

In [ ]:
df_pipelines = hs.pipelines.get_all()
df_pipelines

#### Enter your pipeline id

In [ ]:
pipeline_id = "8432671"

### Get dealstages from pipeline

In [ ]:
df_dealstages = df_pipelines.copy()

# Filter on pipeline
df_dealstages = df_dealstages[df_dealstages.pipeline_id == pipeline_id]

df_dealstages

### Get deals from pipeline

In [ ]:
properties = [
    "hs_object_id",
    "dealname",
    "dealstage",
    "pipeline",
    "createdate",
    "hs_lastmodifieddate",
    "closedate",
    "amount"
]
df_deals = hs.deals.get_all(properties)

# Filter on pipeline
df_deals = df_deals[df_deals.pipeline == pipeline_id].reset_index(drop=True)

df_deals

## Model

### Calculate yearly forecasted

In [ ]:
df_sales = pd.merge(df_deals.drop("pipeline", axis=1),
                    df_dealstages.drop(["pipeline", "pipeline_id", "createdAt", "updatedAt", "archived"], axis=1),
                    left_on="dealstage",
                    right_on="dealstage_id",
                    how="left")
df_sales

In [ ]:
df_sales_c = df_sales.copy()

# Cleaning
df_sales_c.createdate = pd.to_datetime(df_sales_c.createdate)
df_sales_c.hs_lastmodifieddate = pd.to_datetime(df_sales_c.hs_lastmodifieddate)
df_sales_c.closedate = pd.to_datetime(df_sales_c.closedate)
df_sales_c["year"] = df_sales_c.closedate.dt.strftime("%Y")

df_sales_c = df_sales_c[df_sales_c["year"] == str(datetime.now().year)]

df_sales_c["amount"] = df_sales_c["amount"].fillna("0")
df_sales_c.loc[df_sales_c["amount"] == "", "amount"] = "0"

# Formatting
df_sales_c["amount"] = df_sales_c["amount"].astype(float)
df_sales_c["probability"] =  df_sales_c["probability"].astype(float)

# Calc
df_sales_c["forecasted"] = df_sales_c["amount"] * df_sales_c["probability"]

df_sales_c

In [ ]:
df_details = df_sales_c.copy()

# Groupby
to_group = [
    "dealstage_label",
    "probability",
    "displayOrder"
]
to_agg = {
    "amount": "sum",
    "dealname": "count",
    "forecasted": "sum"
}
df_details = df_details.groupby(to_group, as_index=False).agg(to_agg)

# # Format
# df_details["amount"] = df_details["amount"].map(NUMBER_FORMAT.format).str.replace(",", " ")
# df_details["forecasted"] = df_details["forecasted"].map(NUMBER_FORMAT.format).str.replace(",", " ")

# Sort
df_details = df_details.sort_values("displayOrder")

df_details

In [ ]:
forecasted = df_details.forecasted.sum()
forecasted

In [ ]:
won = df_details[df_details["dealstage_label"] == "Cash-in"].forecasted.sum()
won

In [ ]:
weighted = df_details[df_details["dealstage_label"] != "Cash-in"].forecasted.sum()
weighted

In [ ]:
completion_p = forecasted / objective
completion_p

In [ ]:
completion_v = objective - forecasted
completion_v

In [ ]:
DATE_FORMAT = "%Y-%m-%d"

today = datetime.now().strftime(DATE_FORMAT)
today

In [ ]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f} €"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

In [ ]:
def format_pourcentage(num):
    NUMBER_FORMAT = "{:,.0%}"
    num = str(NUMBER_FORMAT.format(num))
    return num

In [ ]:
df = df_details.copy()

details = []

for _, row in df.iterrows():
    # status part
    dealstage = row.dealstage_label
    print(dealstage)
    probability = row.probability
    detail = f"{dealstage} ({format_pourcentage(probability)})"
    
    # amount part
    amount = row.amount
    number = row.dealname
    forecasted_ = row.forecasted
    if (probability < 1 and probability > 0):
        detail = f"{detail}: <ul><li>Amount : {format_number(amount)}</li><li>Number : {number}</li><li>Weighted amount : {format_number(forecasted_)}</li></ul>"
    else:
        detail = f"{detail}: {format_number(amount)}"
        
    details += [detail]

details

In [ ]:
df_inactive = df_sales_c.copy()

df_inactive.hs_lastmodifieddate = pd.to_datetime(df_inactive.hs_lastmodifieddate).dt.strftime(DATE_FORMAT)

df_inactive["inactive_time"] = (datetime.now() - pd.to_datetime(df_inactive.hs_lastmodifieddate, format=DATE_FORMAT)).dt.days
df_inactive.loc[(df_inactive["inactive_time"] > 30, "inactive")] = "inactive"
df_inactive = df_inactive[(df_inactive.inactive == 'inactive') & 
                          (df_inactive.probability != 0.) & 
                          (df_inactive.probability < 1)].sort_values("amount", ascending=False).reset_index(drop=True)

df_inactive

In [ ]:
inactives = []

for _, row in df_inactive[:10].iterrows():
    # status part
    dealname = row.dealname
    dealstage_label = row.dealstage_label
    amount = row.amount
    print(dealname)
    probability = row.probability
    inactive = f"{dealname} ({dealstage_label}): {format_number(amount)}"
    inactives += [inactive]

inactives 

## Output

### Create email

In [ ]:
def email_brief(today,
                forecasted,
                won,
                weighted,
                objective,
                completion_p,
                completion_v,
                details,
                inactives
                ):
    content = {
        'txt_intro': (f"Hi there,<br><br>"
                      f"Here is your weekly sales email as of {today}."),

        'title_1': emailbuilder.text("Overview", font_size="27px", text_align="center", bold=True, underline=True),
        "text_1": emailbuilder.text(f"As of today, your yearly forecasted revenue is <b>{format_number(forecasted)}</b>."),
        "list_1": emailbuilder.list([f"Won : {format_number(won)}",
                                     f"Weighted pipeline : {format_number(weighted)}"]),
        "text_1_1": emailbuilder.text(f"Your yearly objective is {format_number(objective)} ({format_pourcentage(completion_p)} completion)."),
        "text_1_2": emailbuilder.text(f"You need to find <b>{format_number(completion_v)}</b> to reach your goal !"),
        
        'title_2': emailbuilder.text("Sales pipeline", font_size="27px", text_align="center", bold=True, underline=True),
        "list_2": emailbuilder.list(details),

        'title_3': emailbuilder.text("Inactive deals", font_size="27px", text_align="center", bold=True, underline=True),
        'text_3': emailbuilder.text("Here are deals where you need to take actions :"),
        'list_3': emailbuilder.list(inactives),
        'text_3_1': emailbuilder.text("If you need more details, jump into <a href='https://app.hubspot.com/'>Hubspot</a>."),
        
        'title_4': emailbuilder.text("Glossary", bold=True, underline=True),
        'list_4': emailbuilder.list(["Yearly forecasted revenue :  Weighted amount + WON exclude LOST",
                                     "Yearly objective : Input in script",
                                     "Inactive deal : No activity for more than 10 days"]),
        
        'footer_cs': emailbuilder.footer_company(naas=True),
    }
    
    email_content = emailbuilder.generate(display='iframe', **content)
    return email_content

email_content = email_brief(today,
                            forecasted,
                            won,
                            weighted,
                            objective,
                            completion_p,
                            completion_v,
                            details,
                            inactives)

### Send email

In [ ]:
naas.notification.send(email_to,
                       email_subject,
                       email_content)